In [1]:
import pandas as pd
import string
from matplotlib import rc

fontsize = 12

# Make default font serif
rc('font', family=b'serif', size=fontsize)
# Hide legend frame, change size, make only one scatter marker show up
rc('legend', borderpad=0.,labelspacing=0.25, fontsize=(fontsize - 1.), frameon=False, numpoints=1)
# Default line width (e.g. for plot()) should be thinner than 1 point
rc('patch', linewidth=0.75)
rc('lines', linewidth=0.75)

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 30 days


# BW1484 Core I Output

In [3]:
k_ref = 1.00020

### Load and parse the file

In [1]:
import re

# name of the file
out_file='bw1484_small.out' #$1
with open('/home/slayer/Dropbox/Software/Exnihilo/Calculations/bw1484/'+out_file, 'r') as f:

    ## Pairs of output variables and their keywords that always occur and only once:
    vals = {'problem_name' : name, 'eigen_solver' : e_solver, 'eq_set' : eq_set, 
            'num cells I' : I_cells, 'num cells J' : J_cells, 'num cells K' : K_cells,
            'num_blocks_i' : I_blocks, 'num_blocks_j' : J_blocks, 'num_z_blocks' : K_blocks,
            'Pn_order' : Pn, 'azimuthals_octant' : Az, 'polars_octant' : polar, 'DoF' : DoF,             
            'L2_tolerance' : L2_tol, 'k_tolerance' : k_tol, 'tolerance' : tol, 
            'first_group' : g_first, 'last_group' : g_last, 'mesh' : mesh, 'num_sets' : sets,
            'num upscatter groups' : up_groups}
    for line in f:
        for key in vals: 
            if re.search(key, line):
                data = line.split(key)
                vals[key] = data[1].strip() 
                print key, vals[key]
# mg_preconditioner
# relax_count,
# relax_weight,
# num_v_cycles,
# if multilevel_quad_db,
# Pn_order,
# azimuthals_octant,
# polars_octant,
# if upscatter_db; tolerance,



NameError: name 'name' is not defined